# 성능튜닝

## 1.환경준비

### (1) import

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import *

import warnings  # 경고메시지 제외

warnings.filterwarnings(action='ignore')

### (2) 데이터 준비

* 변수설명
    * COLLEGE : 대학 졸업여부
    * INCOME : 연수입
    * OVERAGE : 월평균 초과사용 시간(분)
    * LEFTOVER : 월평균 잔여시간비율(%)
    * HOUSE : 집값
    * HANDSET_PRICE : 스마트폰 가격
    * OVER_15MINS_CALLS_PER_MONTH : 월평균 장기통화(15분이상) 횟수
    * AVERAGE_CALL_DURATION : 평균 통화 시간
    * REPORTED_SATISFACTION : 만족도 설문조사 결과
    * REPORTED_USAGE_LEVEL : 사용도 자가진단 결과
    * CONSIDERING_CHANGE_OF_PLAN : 향후 변경계획 설문조사 결과
    * CHURN : 이탈(번호이동) 여부 (1-이탈, 0-잔류, Target 변수)


In [2]:
# 데이터를 불러옵시다.
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_cust_churn.csv'
data = pd.read_csv(path)
data = data.sample(5000, random_state=2022)
data['CHURN'] = data['CHURN'].map({'LEAVE': 1, 'STAY': 0})
data.head()

,id,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,CHURN
3178,3179,0,119512,51,31,248566,229,5,2,very_sat,very_high,considering,1
14926,14927,1,142144,192,15,774317,581,29,4,unsat,very_little,never_thought,1
15116,15117,1,142308,0,79,306426,497,1,1,sat,little,considering,0
12733,12734,1,113385,0,0,333599,819,1,6,very_unsat,very_high,considering,1
14032,14033,1,90348,209,10,637286,360,26,4,unsat,little,actively_looking_into_it,0


## 2.데이터 준비

### (1) 데이터 정리

In [3]:
drop_cols = ['id']
data.drop(drop_cols, axis=1, inplace=True)

### (2) 데이터분할1 : x, y 나누기

In [4]:
target = 'CHURN'
x = data.drop(target, axis=1)
y = data.loc[:, target]

### (3) NA 조치

### (4) 가변수화

In [5]:
dumm_cols = ['REPORTED_SATISFACTION', 'REPORTED_USAGE_LEVEL', 'CONSIDERING_CHANGE_OF_PLAN']
x = pd.get_dummies(x, columns=dumm_cols, drop_first=True)

### (5) 데이터분할2 : train : validation 나누기

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.3, random_state=20)

### (6) Scaling

In [7]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

## 3.선형모델 튜닝

Logistic Regression : 전진선택법
* 변수를 하나씩 늘려가면서
* AIC를 가장 낮추는 모델 찾기

### (1) 전진선택을 수행할 함수 만들기( **로지스틱 회귀** 용)

In [8]:
# 아래 함수는 로지스틱 회귀를 위한 전진선택법 함수 입니다.
import statsmodels.api as sm


def forward_stepwise_logistic(x_train, y_train):
    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({'step': [], 'feature': [], 'aic': []})

    # 
    for s in range(0, len(features)):
        result = {'step': [], 'feature': [], 'aic': []}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features:
            vars = selected + [f]
            x_tr = x_train[vars]
            model = sm.Logit(y_train, x_tr).fit()
            result['step'].append(s + 1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)

        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop=True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min():
            break
        step_df = pd.concat([step_df, temp], axis=0).reset_index(drop=True)

        # 선택된 변수 제거
        v = temp.loc[0, 'feature'][s]
        features.remove(v)

        selected.append(v)

    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

### (2) 전진선택법 수행

In [9]:
vars, result = forward_stepwise_logistic(x_train, y_train)

Optimization terminated successfully.
         Current function value: 0.693075
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693062
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.683528
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693007
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.684909
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.693042
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.686899
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692565
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692806
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693137
  

* 선택된 변수

In [10]:
vars

['OVERAGE',
 'HOUSE',
 'HANDSET_PRICE',
 'LEFTOVER',
 'REPORTED_SATISFACTION_very_sat',
 'INCOME',
 'REPORTED_SATISFACTION_sat']

In [15]:
result

,step,feature,aic
0,1.0,[OVERAGE],4786.699456
1,1.0,[HOUSE],4796.363859
2,1.0,[OVER_15MINS_CALLS_PER_MONTH],4810.294604
3,1.0,[REPORTED_SATISFACTION_very_sat],4845.064834
4,1.0,[AVERAGE_CALL_DURATION],4849.951663
...,...,...,...
114,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, REPORTED_SATISFACTION_very_unsat]",4442.309949
115,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, CONSIDERING_CHANGE_OF_PLAN_never_thought]",4442.315315
116,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, REPORTED_USAGE_LEVEL_high]",4442.316816
117,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, REPORTED_USAGE_LEVEL_very_little]",4442.448187


In [11]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

### (3) 모델링

* 전체 변수 

In [12]:
m1 = LogisticRegression()
m1.fit(x_train, y_train)
p1 = m1.predict(x_val)

print(accuracy_score(y_val, p1))
print(classification_report(y_val, p1))

0.6333333333333333
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.65      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.63      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



* 전진선택법 변수

In [13]:
m2 = LogisticRegression()
m2.fit(x_train[vars], y_train)
p2 = m2.predict(x_val[vars])

print(accuracy_score(y_val, p2))
print(classification_report(y_val, p2))

0.634
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.66      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.64      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



## 4.하이퍼파라미터 튜닝

### (1) 필요한 함수 불러오기 

In [14]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

### (2) Random Search

① 값의 범위를 지정한다.  
② 모델 선언(시도 횟수 지정)  
③ 모델링(값의 범위 내에서 시도 횟수만큼 랜덤하게 선택해서 시도한다.)  
④ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [16]:
# dictionary형태로 선언
params = {'n_neighbors': range(1, 51), 'metric': ['euclidean', 'manhattan']}
params

{'n_neighbors': range(1, 51), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [17]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_rs = RandomizedSearchCV(model
                              , params  # hyperparameter 범위 지정.
                              , cv=5  # k-fold Cross Validation
                              , n_iter=5  # Random하게 시도할 횟수
                              )

#### ③ 모델링

In [18]:
# 학습 : model이 아니라 model_rs
model_rs.fit(x_train_s, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=5,
                   param_distributions={'metric': ['euclidean', 'manhattan'],
                                        'n_neighbors': range(1, 51)})

In [19]:
# 튜닝 결과
model_rs.cv_results_

{'mean_fit_time': array([0.00040002, 0.00019999, 0.0006    , 0.00040216, 0.00080051]),
 'std_fit_time': array([0.00048992, 0.00039997, 0.0004899 , 0.00049256, 0.00040026]),
 'mean_score_time': array([0.03779979, 0.0382    , 0.03022785, 0.03279085, 0.03799953]),
 'std_score_time': array([0.00074819, 0.00146963, 0.00070494, 0.0017358 , 0.00109589]),
 'param_n_neighbors': masked_array(data=[20, 23, 4, 20, 30],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_metric': masked_array(data=['manhattan', 'manhattan', 'euclidean', 'euclidean',
                    'manhattan'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 20, 'metric': 'manhattan'},
  {'n_neighbors': 23, 'metric': 'manhattan'},
  {'n_neighbors': 4, 'metric': 'euclidean'},
  {'n_neighbors': 20, 'metric': 'euclidean'},
  {'n_neighbors': 30, 'metric': 'manhattan'}],
 'split0_te

In [25]:
print(model_rs.cv_results_)

{'mean_fit_time': array([0.00040002, 0.00019999, 0.0006    , 0.00040216, 0.00080051]), 'std_fit_time': array([0.00048992, 0.00039997, 0.0004899 , 0.00049256, 0.00040026]), 'mean_score_time': array([0.03779979, 0.0382    , 0.03022785, 0.03279085, 0.03799953]), 'std_score_time': array([0.00074819, 0.00146963, 0.00070494, 0.0017358 , 0.00109589]), 'param_n_neighbors': masked_array(data=[20, 23, 4, 20, 30],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_metric': masked_array(data=['manhattan', 'manhattan', 'euclidean', 'euclidean',
                   'manhattan'],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 20, 'metric': 'manhattan'}, {'n_neighbors': 23, 'metric': 'manhattan'}, {'n_neighbors': 4, 'metric': 'euclidean'}, {'n_neighbors': 20, 'metric': 'euclidean'}, {'n_neighbors': 30, 'metric': 'manhattan'}], 'split0_test_score': array([0.60

In [20]:
model_rs.cv_results_['params']

[{'n_neighbors': 20, 'metric': 'manhattan'},
 {'n_neighbors': 23, 'metric': 'manhattan'},
 {'n_neighbors': 4, 'metric': 'euclidean'},
 {'n_neighbors': 20, 'metric': 'euclidean'},
 {'n_neighbors': 30, 'metric': 'manhattan'}]

In [21]:
model_rs.cv_results_['mean_test_score']

array([0.61057143, 0.61      , 0.56771429, 0.58028571, 0.61657143])

In [22]:
# 최적의 파라미터
model_rs.best_params_

{'n_neighbors': 30, 'metric': 'manhattan'}

In [23]:
# 그때의 성능
model_rs.best_score_

0.6165714285714285

In [24]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.58      0.77      0.66       738
           1       0.67      0.45      0.54       762

    accuracy                           0.61      1500
   macro avg       0.63      0.61      0.60      1500
weighted avg       0.63      0.61      0.60      1500



### (3) 실습 : Random Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [42]:
model = DecisionTreeClassifier()

params = {'max_depth': range(1, 11), 'min_samples_leaf': range(10, 101, 10)}

#### ② 모델 선언

In [43]:
model_rs = RandomizedSearchCV(estimator=model,param_distributions=params)

#### ③ 모델링

In [44]:
model_rs.fit(x_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(),
                   param_distributions={'max_depth': range(1, 11),
                                        'min_samples_leaf': range(10, 101, 10)})

In [45]:
model_rs.best_params_

{'min_samples_leaf': 90, 'max_depth': 8}

In [46]:
model_rs.best_score_

0.6954285714285714

In [47]:
pred = model_rs.predict(x_val)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.71      0.67      0.69       738
           1       0.70      0.73      0.71       762

    accuracy                           0.70      1500
   macro avg       0.70      0.70      0.70      1500
weighted avg       0.70      0.70      0.70      1500



### (4) Grid Search

① 값의 범위를 지정한다.  
② 모델링(값의 범위 내에서 모든 조합을 다 시도한다.)  
③ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [ ]:
# dictionary형태로 선언
params = {'n_neighbors': range(3, 31, 2), 'metric': ['euclidean', 'manhattan']}
params

#### ② 모델 선언

In [ ]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_gs = GridSearchCV(model, params, cv=5)

#### ③ 모델링

In [ ]:
# 학습 : model이 아니라 model_rs
model_gs.fit(x_train_s, y_train)

In [ ]:
# 튜닝 결과
model_gs.cv_results_

In [ ]:
model_gs.cv_results_['params']

In [ ]:
model_gs.cv_results_['mean_test_score']

In [ ]:
# 최적의 파라미터
model_gs.best_params_

In [ ]:
# 그때의 성능
model_gs.best_score_

In [ ]:
# best 모델로 예측 및 평가
pred = model_gs.predict(x_val_s)
print(classification_report(y_val, pred))

### (5) 실습 : Grid Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

#### ② 모델 선언

#### ③ 모델링